In [1]:
import pandas as pd
import numpy as np
import os
import datetime as dt
import calendar
from matplotlib import pyplot as plt
import scipy.stats as stats
import glob


from mpl_toolkits.mplot3d import Axes3D  #3Dplot
import seaborn as sns
import japanize_matplotlib

In [85]:
# names にNANの列名を入れておかないとバグる
col_names = ['エリア','発電事業者', '発電所コード', '発電所名', '発電形式', 'ユニット名', '認可出力', '停止区分', '種別',
       '低下量', '停止日時', '復旧見通し', '復旧予定日', '停止原因', '最終更新日時','NAN1']

df_stop =pd.read_csv('../../../卒論関連書類/データ/電源停止データ/outages_20230111143516.csv',encoding='CP932',names=col_names)[1:]
df_stop= df_stop.drop(columns='NAN1')
# 沖縄を除く
df_stop_drop_okinawa =df_stop[df_stop['エリア'] != '沖縄']

# 時間だけKingから持ってくる
files = glob.glob('../../../卒論関連書類/データ/king_history/king_history1/*')
version = len(files)-1
df_king =pd.read_csv(f'../../../卒論関連書類/データ/king_history/king_history1/king{version}.csv',converters={'DateTime':pd.to_datetime})


In [260]:
# df_stop_king を作成する

# names にNANの列名を入れておかないとバグる
col_names = ['エリア','発電事業者', '発電所コード', '発電所名', '発電形式', 'ユニット名', '認可出力', '停止区分', '種別',
       '低下量', '停止日時', '復旧見通し', '復旧予定日', '停止原因', '最終更新日時','NAN1']

df_stop =pd.read_csv('../../../卒論関連書類/データ/電源停止データ/outages_20230111143516.csv',encoding='CP932',names=col_names)[1:]
df_stop= df_stop.drop(columns='NAN1')
# 沖縄を除く
df_stop_drop_okinawa =df_stop[df_stop['エリア'] != '沖縄']

# 時間だけKingから持ってくる
files = glob.glob('../../../卒論関連書類/データ/king_history/king_history1/*')
version = len(files)-1
df_king =pd.read_csv(f'../../../卒論関連書類/データ/king_history/king_history1/king{version}.csv',converters={'DateTime':pd.to_datetime})

df_stop['停止・低下ユニット数'] = 1
df_stop.loc[df_stop['低下量'].isna(),['低下量']] = df_stop[df_stop['低下量'].isna()]['認可出力']


# 停止日時をTimeStampに変換
stop_str_datelist = df_stop['停止日時'].values
str_datelist = stop_str_datelist
stop_datetime_list = []
for str_date in str_datelist:

    stop_datetime_list.append(dt.datetime.strptime(str_date,'%Y/%m/%d %H:%M'))

    if stop_datetime_list[-1].year>2050:
        stop_datetime_list[-1] = dt.datetime.strptime('2050/12/31 23:30','%Y/%m/%d %H:%M')
        # print('yes',stop_datetime_list[-1],len(stop_datetime_list))

df_stop['停止日時'] = stop_datetime_list

# 復旧日時をTimeStampに変換,復旧日時では終日止まっていることとしている。
df_stop['復旧予定日'] =df_stop['復旧予定日'].fillna('NONE')
start_str_datelist = df_stop['復旧予定日'].values
str_datelist = start_str_datelist
start_datetime_list = []

for str_date in str_datelist:
    if str_date=='NONE':
        start_datetime = dt.datetime.strptime('2060/12/31 23:30','%Y/%m/%d %H:%M')
    else:
        start_datetime = dt.datetime.strptime(f'{str_date} 23:30','%Y/%m/%d %H:%M')



    start_datetime_list.append(start_datetime)
    # if stop_datetime_list[-1].year>2061:
        # start_datetime_list[-1] = dt.datetime.strptime('2060/12/31 23:30','%Y/%m/%d %H:%M')
        # print('yes',start_datetime_list[-1],len(stop_datetime_list))
new_start_datetimelist =[]
for date in start_datetime_list:
    if date.year >2060:
        new_start_datetimelist.append(dt.datetime.strptime('2060/12/31 23:30','%Y/%m/%d %H:%M'))
    else:
        new_start_datetimelist.append(date)        


df_stop['復旧予定日'] = new_start_datetimelist


# まとめるファイルの初期設定
df_stop_king = pd.DataFrame(columns=['DateTime','認可出力合計','低下量合計','停止・低下ユニット数合計'])
df_stop_king['DateTime'] =df_king['DateTime']
# それぞれのパラメータの値を０にする。その後足し算ができるように
df_stop_king = df_stop_king.fillna(0)

for num in range(len(df_stop)):

    df_select = df_stop[num:num+1]
    start_point =df_select['停止日時'].values[0]
    end_point = df_select['復旧予定日'].values[0]

    col_names_stop_king = ['認可出力','低下量','停止・低下ユニット数']
    try:
        for k in range(len(col_names_stop_king)):
            df_stop_king.loc[(df_stop_king['DateTime']>=start_point) &(df_stop_king['DateTime']<=end_point),[f'{col_names_stop_king[k]}合計']] = df_stop_king[(df_stop_king['DateTime']>=start_point) &(df_stop_king['DateTime']<=end_point)][f'{col_names_stop_king[k]}合計'].values + float(df_select[f'{col_names_stop_king[k]}'].values[0])
    except:
        df = df_select
        print(df_select)


/var/folders/c5/xmh5z7yd191cl82bdh1n_kzc0000gn/T/ipykernel_30786/3726978696.py:77: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_stop_king.loc[(df_stop_king['DateTime']>=start_point) &(df_stop_king['DateTime']<=end_point),[f'{col_names_stop_king[k]}合計']] = df_stop_king[(df_stop_king['DateTime']>=start_point) &(df_stop_king['DateTime']<=end_point)][f'{col_names_stop_king[k]}合計'].values + float(df_select[f'{col_names_stop_king[k]}'].values[0])
/var/folders/c5/xmh5z7yd191cl82bdh1n_kzc0000gn/T/ipykernel_30786/3726978696.py:77: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.is

In [266]:
df_stop_king.columns = ['DateTime', '認可出力合計(kW)', '低下量合計(kW)', '停止・低下ユニット数合計']

In [267]:
df_stop_king

,DateTime,認可出力合計(kW),低下量合計(kW),停止・低下ユニット数合計
0,2018-04-01 00:00:00,88445020.0,88445020.0,177.0
1,2018-04-01 00:30:00,55206600.0,55206600.0,98.0
2,2018-04-01 01:00:00,55206600.0,55206600.0,98.0
3,2018-04-01 01:30:00,55206600.0,55206600.0,98.0
4,2018-04-01 02:00:00,55206600.0,55206600.0,98.0
...,...,...,...,...
83672,2023-01-05 21:30:00,124166830.0,114507230.0,293.0
83673,2023-01-05 22:00:00,153219770.0,148365820.0,387.0
83674,2023-01-05 22:30:00,128895980.0,119756777.0,301.0
83675,2023-01-05 23:00:00,117378280.0,108640230.0,280.0


In [268]:
df_stop_king.to_csv('../../../卒論関連書類/データ/抽出＿分析データ/電源停止合算/電源停止合算2.csv')

In [258]:
df_stop.loc[df_stop['低下量'].isna(),['低下量']] = df_stop[df_stop['低下量'].isna()]['認可出力']

In [259]:
df_stop

,エリア,発電事業者,発電所コード,発電所名,発電形式,ユニット名,認可出力,停止区分,種別,低下量,停止日時,復旧見通し,復旧予定日,停止原因,最終更新日時,停止・低下ユニット数
1,九州,九州電力株式会社,95062,松浦発電所,火力（石炭）,２号機,1000000,出力低下,低下・設備故障,500000,2022-12-07 18:55:00,あり,2023-01-27 23:30:00,"ボイラー付属設備点検\r\n最大低下量500,000kW\r\n最小低下量500,000kW...",2023/01/11 14:28,1
2,中国,ENEOS株式会社,4009774203,麻里布 第２発電所,その他,Ｇ６,149000,計画停止,停止・定期検査等,149000,2023-01-23 11:00:00,あり,2023-03-25 23:30:00,定期補修,2023/01/11 13:26,1
3,中部,株式会社JERA,42299,川越火力発電所,火力（ガス）,４－２号機,243000,計画外停止,停止・その他,243000,2023-01-11 13:04:00,なし,2060-12-31 23:30:00,計測関連設備,2023/01/11 13:10,1
4,九州,九州電力株式会社,95012,天山発電所,水力,２号機,300000,計画停止,停止・定期検査等,300000,2023-01-16 12:00:00,あり,2023-01-16 23:30:00,定期点検,2023/01/11 12:08,1
5,九州,九州電力株式会社,95012,天山発電所,水力,１号機,300000,計画停止,停止・定期検査等,300000,2023-01-16 12:00:00,あり,2023-01-16 23:30:00,定期点検,2023/01/11 12:07,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27777,中国,中国電力株式会社,78301,俣野川発電所,水力,４号機,300000,計画停止,NaN,300000,2016-04-21 10:00:00,あり,2016-04-21 23:30:00,NaN,2016/04/12 13:23,1
27778,中国,中国電力株式会社,78301,俣野川発電所,水力,２号機,300000,計画停止,NaN,300000,2016-04-20 13:00:00,あり,2016-04-20 23:30:00,NaN,2016/04/12 13:22,1
27779,中国,中国電力株式会社,78301,俣野川発電所,水力,１号機,300000,計画停止,NaN,300000,2016-04-20 10:00:00,あり,2016-04-20 23:30:00,NaN,2016/04/12 13:22,1
27780,中国,中国電力株式会社,76301,玉島発電所第１号機,火力（ガス）,１号機,350000,計画停止,NaN,350000,2016-03-01 00:00:00,あり,2016-05-17 23:30:00,NaN,2016/04/12 13:17,1


In [228]:
df

,エリア,発電事業者,発電所コード,発電所名,発電形式,ユニット名,認可出力,停止区分,種別,低下量,停止日時,復旧見通し,復旧予定日,停止原因,最終更新日時,停止・低下ユニット数
12397,関西,関西電力株式会社,63D03,大河内発電所,水力,１号機,320000,計画停止,NaN,NaN,2050-12-31 23:30:00,あり,9999-02-16 23:30:00,NaN,2020/11/27 14:21,1


In [231]:
for date in stop_datetime_list:
    if date.year>2060:
        
        print('yes')
    else:
        pass

In [233]:
new_start_datetimelist =[]
for date in start_datetime_list:
    if date.year >2060:
        new_start_datetimelist.append(dt.datetime.strptime('2060/12/31 23:30','%Y/%m/%d %H:%M'))
    else:
        new_start_datetimelist.append(date)        


In [234]:
new_start_datetimelist

[datetime.datetime(2023, 1, 27, 23, 30),
 datetime.datetime(2023, 3, 25, 23, 30),
 datetime.datetime(2060, 12, 31, 23, 30),
 datetime.datetime(2023, 1, 16, 23, 30),
 datetime.datetime(2023, 1, 16, 23, 30),
 datetime.datetime(2023, 1, 12, 23, 30),
 datetime.datetime(2060, 12, 31, 23, 30),
 datetime.datetime(2023, 6, 30, 23, 30),
 datetime.datetime(2023, 1, 14, 23, 30),
 datetime.datetime(2023, 1, 10, 23, 30),
 datetime.datetime(2023, 1, 20, 23, 30),
 datetime.datetime(2023, 1, 10, 23, 30),
 datetime.datetime(2023, 1, 11, 23, 30),
 datetime.datetime(2023, 11, 11, 23, 30),
 datetime.datetime(2023, 1, 11, 23, 30),
 datetime.datetime(2023, 1, 10, 23, 30),
 datetime.datetime(2023, 1, 8, 23, 30),
 datetime.datetime(2023, 1, 30, 23, 30),
 datetime.datetime(2023, 1, 23, 23, 30),
 datetime.datetime(2023, 1, 10, 23, 30),
 datetime.datetime(2023, 1, 10, 23, 30),
 datetime.datetime(2023, 1, 9, 23, 30),
 datetime.datetime(2023, 1, 12, 23, 30),
 datetime.datetime(2060, 12, 31, 23, 30),
 datetime.date

In [199]:
df_stop[df_stop['予定日']>2020]

TypeError: '>' not supported between instances of 'datetime.datetime' and 'int'

In [219]:
pd.to_datetime(df_stop['停止日時'])

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 9999-03-22 23:59:00 present at position 122

In [ ]:


start_datetime = dt.datetime.strptime(f'{str_date} 23:30','%Y/%m/%d %H:%M')

In [182]:
col_names_stop_king = ['認可出力','低下量','停止・低下ユニット数']
k=0
df_stop_king.loc[(df_stop_king['DateTime']>=start_point) &(df_stop_king['DateTime']<=end_point),[f'{col_names_stop_king[k]}合計']] = df_stop_king[(df_stop_king['DateTime']>=start_point) &(df_stop_king['DateTime']<=end_point)][f'{col_names_stop_king[k]}合計'].values + float(df_select[f'{col_names_stop_king[k]}'].values[0])

In [ ]:
df

In [47]:
df_stop_drop_okinawa.columns

Index(['エリア', '発電事業者', '発電所コード', '発電所名', '発電形式', 'ユニット名', '認可出力', '停止区分', '種別',
       '低下量', '停止日時', '復旧見通し', '復旧予定日', '停止原因', '最終更新日時', 'Area'],
      dtype='object')

In [155]:
df_stop['停止・低下ユニット数'] = 1
df_stop_king = pd.DataFrame(columns=['DateTime','認可出力合計','低下量合計','停止・低下ユニット数合計'])

In [156]:
df_stop_king['DateTime'] =df_king['DateTime']
# それぞれのパラメータの値を０にする。その後足し算ができるように
df_stop_king = df_stop_king.fillna(0)

In [94]:
df_stop.columns

Index(['エリア', '発電事業者', '発電所コード', '発電所名', '発電形式', 'ユニット名', '認可出力', '停止区分', '種別',
       '低下量', '停止日時', '復旧見通し', '復旧予定日', '停止原因', '最終更新日時', 'unit_num'],
      dtype='object')

In [105]:
# 停止日時をTimeStampに変換
stop_str_datelist = df_stop['停止日時'].values
str_datelist = stop_str_datelist
stop_datetime_list = []
for str_date in str_datelist:
    stop_datetime_list.append(dt.datetime.strptime(str_date,'%Y/%m/%d %H:%M'))

df_stop['停止日時'] = stop_datetime_list

In [135]:
# 復旧日時をTimeStampに変換,復旧日時では終日止まっていることとしている。
df_stop['復旧予定日'] =df_stop['復旧予定日'].fillna('NONE')
start_str_datelist = df_stop['復旧予定日'].values
str_datelist = start_str_datelist
start_datetime_list = []

for str_date in str_datelist:
    if str_date=='NONE':
        start_datetime = dt.datetime.strptime('9999/12/31 23:30','%Y/%m/%d %H:%M')
    else:
        start_datetime = dt.datetime.strptime(f'{str_date} 23:30','%Y/%m/%d %H:%M')
    start_datetime_list.append(start_datetime)

df_stop['復旧予定日'] = start_datetime_list


In [142]:
num = 0
df_select = df_stop[num:num+1]
start_point =df_select['停止日時'].values[0]
end_point = df_select['復旧予定日'].values[0]

In [ ]:
df_stop_king.loc[(df_stop_king['DateTime']>=start_point) &(df_stop_king['DateTime']<=end_point),['認可出力合計']] = df_stop_king[(df_stop_king['DateTime']>=start_point) &(df_stop_king['DateTime']<=end_point)]['認可出力合計'].values + float(df_select['認可出力'].values[0])

In [157]:
df_stop_king

,DateTime,認可出力合計,低下量合計,停止・低下ユニット数合計
0,2018-04-01 00:00:00,0,0,0
1,2018-04-01 00:30:00,0,0,0
2,2018-04-01 01:00:00,0,0,0
3,2018-04-01 01:30:00,0,0,0
4,2018-04-01 02:00:00,0,0,0
...,...,...,...,...
83672,2023-01-05 21:30:00,0,0,0
83673,2023-01-05 22:00:00,0,0,0
83674,2023-01-05 22:30:00,0,0,0
83675,2023-01-05 23:00:00,0,0,0


In [175]:
df_stop_king.loc[(df_stop_king['DateTime']>=start_point) &(df_stop_king['DateTime']<=end_point),['認可出力合計']] = df_stop_king[(df_stop_king['DateTime']>=start_point) &(df_stop_king['DateTime']<=end_point)]['認可出力合計'].values + float(df_select['認可出力'].values[0])

In [176]:
df_stop_king

,DateTime,認可出力合計,低下量合計,停止・低下ユニット数合計,認,可,出,力,合,計
0,2018-04-01 00:00:00,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-04-01 00:30:00,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-04-01 01:00:00,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-04-01 01:30:00,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-04-01 02:00:00,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
83672,2023-01-05 21:30:00,1000000,0,0,NaN,NaN,NaN,NaN,NaN,NaN
83673,2023-01-05 22:00:00,1000000,0,0,NaN,NaN,NaN,NaN,NaN,NaN
83674,2023-01-05 22:30:00,1000000,0,0,NaN,NaN,NaN,NaN,NaN,NaN
83675,2023-01-05 23:00:00,1000000,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [173]:
df_stop_king[(df_stop_king['DateTime']>=start_point) &(df_stop_king['DateTime']<=end_point)]['認可出力合計'].values +float(df_select['認可出力'].values[0])

array([1000000., 1000000., 1000000., ..., 1000000., 1000000., 1000000.])

In [167]:
df_select['認可出力'].values[0]

'1000000'

In [134]:
start_datetime_list[-1]

datetime.datetime(2016, 4, 27, 23, 30)

In [111]:
start_str_datelist = df_stop['復旧予定日'].values

In [130]:
df_stop['復旧予定日'].fillna('NONE')

1        2023/01/27
2        2023/03/25
3              NONE
4        2023/01/16
5        2023/01/16
            ...    
27777    2016/04/21
27778    2016/04/20
27779    2016/04/20
27780    2016/05/17
27781    2016/04/27
Name: 復旧予定日, Length: 27781, dtype: object

In [128]:
str_date

'2023/01/27'

In [129]:
if np.isnan(start_str_datelist[1]):
    print('yes')

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [122]:
type(start_str_datelist[2])

float

In [98]:
# 復旧予定日は終日止まっている想定とする。
start df_stop['復旧予定日'].values[0]

'2023/01/27'

In [103]:
datetime_list

[datetime.datetime(2022, 12, 7, 18, 55),
 datetime.datetime(2023, 1, 23, 11, 0),
 datetime.datetime(2023, 1, 11, 13, 4),
 datetime.datetime(2023, 1, 16, 12, 0),
 datetime.datetime(2023, 1, 16, 12, 0),
 datetime.datetime(2023, 1, 10, 20, 10),
 datetime.datetime(2023, 1, 11, 6, 15),
 datetime.datetime(2023, 3, 1, 0, 0),
 datetime.datetime(2023, 1, 11, 6, 0),
 datetime.datetime(2023, 1, 10, 1, 55),
 datetime.datetime(2022, 12, 29, 17, 0),
 datetime.datetime(2023, 1, 10, 14, 38),
 datetime.datetime(2023, 1, 11, 0, 0),
 datetime.datetime(2023, 1, 11, 0, 0),
 datetime.datetime(2023, 1, 5, 17, 0),
 datetime.datetime(2023, 1, 9, 12, 27),
 datetime.datetime(2023, 1, 8, 6, 35),
 datetime.datetime(2023, 1, 30, 10, 0),
 datetime.datetime(2023, 1, 23, 9, 0),
 datetime.datetime(2023, 1, 10, 0, 0),
 datetime.datetime(2023, 1, 10, 0, 0),
 datetime.datetime(2023, 1, 9, 10, 40),
 datetime.datetime(2023, 1, 9, 4, 55),
 datetime.datetime(2023, 1, 9, 1, 30),
 datetime.datetime(2023, 1, 8, 20, 0),
 datetime

In [169]:

col_names = ['認可出力合計', '低下量合計', '停止・低下ユニット数合計'] 

df_stop_king[(df_stop_king['DateTime']>=start_point) &(df_stop_king['DateTime']<=end_point)]['認可出力合計'] =  df_stop_king[(df_stop_king['DateTime']>=start_point) &(df_stop_king['DateTime']<=end_point)]['認可出力合計'].values + float(df_select['認可出力'].values[0])

/var/folders/c5/xmh5z7yd191cl82bdh1n_kzc0000gn/T/ipykernel_30786/3492072704.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stop_king[(df_stop_king['DateTime']>=start_point) &(df_stop_king['DateTime']<=end_point)]['認可出力合計'] =  df_stop_king[(df_stop_king['DateTime']>=start_point) &(df_stop_king['DateTime']<=end_point)]['認可出力合計'].values + float(df_select['認可出力'].values[0])


In [79]:
# 停止区分は出力低下、計画停止、計画外停止のみ
df_stop['停止区分'].drop_duplicates()

1     出力低下
2     計画停止
3    計画外停止
Name: 停止区分, dtype: object

In [91]:
df_stop

,エリア,発電事業者,発電所コード,発電所名,発電形式,ユニット名,認可出力,停止区分,種別,低下量,停止日時,復旧見通し,復旧予定日,停止原因,最終更新日時,unit_num
1,九州,九州電力株式会社,95062,松浦発電所,火力（石炭）,２号機,1000000,出力低下,低下・設備故障,500000,2022/12/07 18:55,あり,2023/01/27,"ボイラー付属設備点検\r\n最大低下量500,000kW\r\n最小低下量500,000kW...",2023/01/11 14:28,1
2,中国,ENEOS株式会社,4009774203,麻里布 第２発電所,その他,Ｇ６,149000,計画停止,停止・定期検査等,NaN,2023/01/23 11:00,あり,2023/03/25,定期補修,2023/01/11 13:26,1
3,中部,株式会社JERA,42299,川越火力発電所,火力（ガス）,４－２号機,243000,計画外停止,停止・その他,NaN,2023/01/11 13:04,なし,NaN,計測関連設備,2023/01/11 13:10,1
4,九州,九州電力株式会社,95012,天山発電所,水力,２号機,300000,計画停止,停止・定期検査等,NaN,2023/01/16 12:00,あり,2023/01/16,定期点検,2023/01/11 12:08,1
5,九州,九州電力株式会社,95012,天山発電所,水力,１号機,300000,計画停止,停止・定期検査等,NaN,2023/01/16 12:00,あり,2023/01/16,定期点検,2023/01/11 12:07,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27777,中国,中国電力株式会社,78301,俣野川発電所,水力,４号機,300000,計画停止,NaN,NaN,2016/04/21 10:00,あり,2016/04/21,NaN,2016/04/12 13:23,1
27778,中国,中国電力株式会社,78301,俣野川発電所,水力,２号機,300000,計画停止,NaN,NaN,2016/04/20 13:00,あり,2016/04/20,NaN,2016/04/12 13:22,1
27779,中国,中国電力株式会社,78301,俣野川発電所,水力,１号機,300000,計画停止,NaN,NaN,2016/04/20 10:00,あり,2016/04/20,NaN,2016/04/12 13:22,1
27780,中国,中国電力株式会社,76301,玉島発電所第１号機,火力（ガス）,１号機,350000,計画停止,NaN,NaN,2016/03/01 00:00,あり,2016/05/17,NaN,2016/04/12 13:17,1


In [56]:
df_stop =pd.read_csv('../../../卒論関連書類/データ/電源停止データ/outages_20230111143516.csv',encoding='CP932')

In [ ]:
df